## Week3 Peer Graded Assignment: Massimo - SECOND PART

### Install the packages needed and import

In [9]:
#!pip install BeautifulSoup4
#!pip install geocoder

In [10]:
#import needed packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder

### Read the wikipedia page and import in Beautiful Soap object

In [11]:
# make url request and create soup for better data reading
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = r.text
soup = BeautifulSoup(html, 'html.parser')

### Read the table from Soup object

In [12]:
tc=[]
table=soup.find('table')
tab = table.findAll('td')
for row in tab:
    cell = {}
    if row.span.text=='Not assigned':
        # Neglect the "NOT ASSIGNED"
        pass
    else:
        # Assign to postal code the first 3 characters of the row
        cell['PostalCode'] = row.p.text[:3]
        # generate an array with Borough and Neighborhoods, splitting after the parenthesis
        row_array = (row.span.text).split('(')
        # Assign to Borough the first element in the array
        cell['Borough'] = row_array[0]
        
        # remove last closed parenthesis in the Neighborhood
        Neigh = (row_array[1]).strip(')')
        
        # in case of more than one Neighborhoods, put them separated by a comma
        Neigh = Neigh.replace(' /',',')
        cell['Neighborhood'] = Neigh
        tc.append(cell)

### Import in the Pandas Dataframe

In [13]:
df=pd.DataFrame(tc)

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### Print info about the dataframe

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [15]:
df.shape

(103, 3)

In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


## Import what is needed for clustering and visualization

In [17]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

### Read the file with LAT/LONG into a new DF

In [18]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Search for LAT/LONG in the new file and associate to the initial Dataframe

In [19]:
lat =[]
long = []
for count in range(df.shape[0]):
    ii=df_geo['Postal Code'] == df['PostalCode'][count]
    lat.append(df_geo[ii].iat[0,1])
    long.append(df_geo[ii].iat[0,2])

In [20]:
df['Latitude']=lat
df['Longitude']=long
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [21]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
# create map of New York using latitude and longitude values
map_t = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)    
map_t


## ASSINGN THE NEEDED DATAFRAME TO "toronto_data" AND VISUALIZE IT

In [24]:
toronto_data=df
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
